<a href="https://colab.research.google.com/github/tanakt-hub/Test/blob/main/JPMA_2022_TF_1_1_demo_(3)BERT%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続

In [ ]:
# データ受け渡しのためにGoogle Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データ保存ディレクトリの指定
datadir = '/content/drive/MyDrive/datadir/'

Mounted at /content/drive


## データのロード

In [ ]:
# 分かち書き済みのテキストとベクトル化したデータを読み込み
import pickle

with open(datadir + 'datadic.pkl', 'rb') as f:
  datadic = pickle.load(f)

## pip install

In [ ]:
!pip install transformers
!pip install mecab-python3 fugashi 
!pip install jaconv neologdn
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 9.0 MB/s 
     |████████████████████████████████| 182 kB 71.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 6.6 MB/s 
     |████████████████████████████████| 583 kB 30.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.5 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15566 sha256=00fd04bcc1f54e6e899156bb0d1f212731eb3bcee8adfaac50d2a2decb750f5d
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172939 sha256=f4a

## データロードと環境構築

In [ ]:
import jaconv
import unicodedata
import neologdn
import re
import torch

# デバイス設定
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [ ]:
# MeCabとNEologdの設定
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

# 環境変数でmecabrcの場所を指定
import os
os.environ['MECABRC'] = "/etc/mecabrc" 

# NEologdの展開場所を取得
import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書のダウンロードと設定
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 5 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-upd

In [ ]:
# BERTのモデル名を指定（東北大モデル：ライセンスCC BY-SA 3.0）
# 別の事前学習モデル（UTHBERTやMEDBERTjpなど）を使用する場合は展開したディレクトリを指定する
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# JapaneseTokenizer用のMeCabパラメータを指定（学習データは既に分かち書き済みのデータのため Fune tuning には不要）
MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}

# 別の事前学習モデルを使用する場合は対象モデルのLengthに注意（UTHBERT：512、MEDBERTjp：128）
MaxSeqLen = 512

# 学習の準備

## 学習パラメータの設定

In [ ]:
# 学習パラメータの設定
# バッチサイズの設定
BATCH_SIZE = 32

# Learning Rateno設定
LEAENING_RATE = 1e-6

# エポック数の設定
N_EPOCHS = 10

## Tokenizerの設定

In [ ]:
from transformers import BertJapaneseTokenizer

# トークナイザの設定
tokenizer = BertJapaneseTokenizer.from_pretrained(
    model_name,
    word_tokenizer_type = "mecab",
    mecab_kwargs = MeCabDic
    )

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

## 学習データのトークナイズ

In [ ]:
# 対象データを random seed = 0 で分割
# 分割した Train データで Fine-tuning を実施
from sklearn.model_selection import train_test_split
text, X_test,label, y_test = train_test_split(datadic['wakati'], datadic['flg'], test_size=0.15, random_state = 0)


# 最大トークン数の確認
# これを超えるトークンは切り落とされるので注意
model_seq_len = MaxSeqLen

# 学習データの最大トークン数を取得
max_tk = 0
for i, chktoken in enumerate(text):
  tk = tokenizer.tokenize(chktoken)
  if len(tk) > max_tk:
    max_tk = len(tk)
    id = i

# 学習データの最大長に合わせて Length を設定
max_len = max_tk + 2 if max_tk + 2 < model_seq_len else model_seq_len

# 最大長データの確認
tokchk = tokenizer.encode_plus(
    text[id],
    add_special_tokens = True,        # スペシャルトークンの追加
    truncation = True,                # モデル定義長を超える場合の切り捨て
    max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
    padding = 'max_length',           # 最大長までPADDING
    return_overflowing_tokens = True, # 切り捨てられたトークンを返す
    num_truncated_tokens = True       # 切り捨てられたトークン数を返す
    )

print("最大トークン数:", max_tk)
print("*** 最大トークン数に分割されるテキスト ***")
print("  ", text[id])
print("*** BERTに入力されるテキスト ***")
print("  ", tokenizer.decode(tokchk['input_ids']))
print("*** 切り捨てられるテキスト ***")
print("  ", tokenizer.decode(tokchk['overflowing_tokens']))


Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 129
*** 最大トークン数に分割されるテキスト ***
   血液検査 正球性 貧血 red cell count 0 0 x 0 0 uL reference range 0 0 0 0 x 0 0 uL hemoglobin 0 0 g dL 0 0 0 0 g dL hematocrit 0 0% 0 0% 0 0% 正常 血小板 数 0 x 0 0 uL 0 0 x 0 0 uL prothrombin time 0 0 s 0 0 0 0 s activated partial thromboplastin time 0 0 s 0 0 0 0 s 判明 する た
*** BERTに入力されるテキスト ***
   [CLS] 血液検査 正球性 貧血 red cell count 0 0 x 0 0 uL reference range 0 0 0 0 x 0 0 uL hemoglobin 0 0 g dL 0 0 0 0 g dL hematocrit 0 0% 0 0% 0 0% 正常 血小板 数 0 x 0 0 uL 0 0 x 0 0 uL prothrombin time 0 0 s 0 0 0 0 s activated partial thromboplastin time 0 0 s 0 0 0 0 s 判明 する た [SEP]
*** 切り捨てられるテキスト ***
   


In [ ]:


# トークナイズ処理
# 必要なToken IDとAttentionマスクを取得
token_ids = []
attention_masks = []

for t in text:
  tknzd = tokenizer.encode_plus(
      t,
      add_special_tokens = True,        # スペシャルトークンの追加
      truncation = True,                # モデル定義長を超える場合の切り捨て
      max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
      padding = 'max_length'            # 最大長までPADDING
      )
  token_ids.append(tknzd['input_ids'])
  attention_masks.append(tknzd['attention_mask'])

# tensor型に変換
token_ids_t = torch.tensor(token_ids)
attention_masks_t = torch.tensor(attention_masks)
labels_t = torch.tensor(label)

# 変換結果の確認
x = 0
print(tokenizer.tokenize(text[x]))
print(token_ids_t[x])
print(attention_masks_t[x])
print(labels_t[x])

['既', '##往', '##歴', '##なし']
tensor([    2,  1844, 29969, 29128,  5695,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0, 

## データセットとデータローダーの作成

In [ ]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.dataset import Subset
from sklearn.model_selection import StratifiedKFold

# 全学習データをデータセット化
dataset = TensorDataset(token_ids_t, attention_masks_t, labels_t)

# Stratified k-Fold
k = 5
kf = StratifiedKFold(n_splits=k)

train_sets={}
valid_sets={}
for _fold, (train_index, valid_index) in enumerate(kf.split(dataset,dataset[:][-1])):
  train_dataset = Subset(dataset, train_index)
  valid_dataset = Subset(dataset, valid_index)

  train_dataloader = DataLoader(
      train_dataset,
      batch_size = BATCH_SIZE,
      shuffle = True,     # ランダムで取得するか否か
      drop_last = True    # バッチ数に満たないラストデータを落とすか否か
      )
  valid_dataloader = DataLoader(
      valid_dataset,
      batch_size = BATCH_SIZE,
      shuffle = False,
      drop_last = False
      )
  
  tname = "train_" + str(_fold)
  vname = "valid_" + str(_fold)
  train_sets[tname] = train_dataloader
  valid_sets[vname] = valid_dataloader

  print('*** Fold ', _fold, '***')
  print('学習データ数：', len(train_dataset))
  print('検証データ数: ', len(valid_dataset))
  print('Sum of Pos in Val: ', sum(valid_dataset[:][-1]), '\n')


*** Fold  0 ***
学習データ数： 2942
検証データ数:  736
Sum of Pos in Val:  tensor(387) 

*** Fold  1 ***
学習データ数： 2942
検証データ数:  736
Sum of Pos in Val:  tensor(388) 

*** Fold  2 ***
学習データ数： 2942
検証データ数:  736
Sum of Pos in Val:  tensor(388) 

*** Fold  3 ***
学習データ数： 2943
検証データ数:  735
Sum of Pos in Val:  tensor(387) 

*** Fold  4 ***
学習データ数： 2943
検証データ数:  735
Sum of Pos in Val:  tensor(387) 



# Fine-tuningの実行と確認

## 実行関数の定義

In [ ]:
import pandas as pd
import numpy as np

# 訓練パート関数の定義
def train(train_dataloader, model, optimizer, device):
  train_losses = []
  model.train()     # 訓練モード
  optimizer.zero_grad()
  for n_iter, d in enumerate(train_dataloader):
    outputs = model(
      d[0].to(device),                    # input_ids_t
      attention_mask = d[1].to(device),   # attention_masks_t
      labels = d[2].to(device),           # labels_t
      token_type_ids=None
      )
    
    loss = outputs.loss # BertForSequenceClassificationから損失関数CrossEntropyLossの結果を受け取る
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 勾配クリッピング

    optimizer.step()
    optimizer.zero_grad()

    train_losses.append(loss.item())

  return train_losses

# 検証パート関数の定義
def validation(val_dataloader, model, device):
  val_losses = []
  logits     = []
  labels     = []
  inputs     = []

  model.eval()    # 検証モード
  for n_iter, d in enumerate(val_dataloader):
    with torch.no_grad():
      outputs = model(
        d[0].to(device),                    # input_ids_t
        attention_mask = d[1].to(device),   # attention_masks_t
        labels = d[2].to(device),           # labels_t
        token_type_ids=None
        )

    loss = outputs.loss # BertForSequenceClassificationから損失関数CrossEntropyLossの結果を受け取る
    val_losses.append(loss.item())
    
    logits += outputs.logits.sigmoid().cpu().tolist()
    inputs += d[0].tolist()
    labels += d[2].tolist()

  # Predictionの結果をDataFrameで返す
  val_res = pd.DataFrame(logits, columns=['logit0', 'logit1'])
  val_pred = np.argmax(val_res.values, axis=1).tolist()
  val_res['label'] = labels
  val_res['pred']  = val_pred
  val_res['input_ids']  = inputs
  
  return val_losses, val_res

## 実行とモデルの保存

In [ ]:
from transformers import  AdamW, BertForSequenceClassification
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm


# ミニバッチごとのLossを格納する変数を定義
train_losses = []
val_losses = []

# 学習の実施
for i in range(k):

  # BertForSequenceClassificationに事前学習モデルをロード
  model = BertForSequenceClassification.from_pretrained(
      model_name,
      num_labels = 2,                # Binary classification
      output_attentions = False,     # Attentionの出力
      output_hidden_states = False,  # 隠れ層の出力
      )
  
  # 最適化手法の設定
  optimizer = torch.optim.AdamW(model.parameters(), lr=LEAENING_RATE)

  # モデルをGPUへ転送
  model.to(device)
  print(device)

  print('*** CV', i, 'started')
  for epoch in tqdm(range(N_EPOCHS), total = N_EPOCHS):

    train_ = train(train_sets["train_"+str(i)], model, optimizer, device)
    loss, val_res = validation(valid_sets["valid_"+str(i)], model, device)

    '''
    # Epochごとの混同行列を表示させる場合はこのコメントアウト部分を実行
    cm = confusion_matrix(val_res['label'].tolist(), val_res['pred'].tolist())
    cm_df = pd.DataFrame(cm,columns=['Predicted Neg', 'Predicted Pos'], index=['Actual Neg', 'Actual Pos'])
    display(cm_df)
    '''

    print('  epoch', epoch, 'total loss :', sum(loss))

    train_losses += train_
    val_losses += loss

 
  '''
  # 教師ラベルと異なる予測となった一覧を表示させる場合はこのコメントアウト部分を実行
  val_res['Text'] = [t.strip('[CLS] [SEP] [PAD]') for t in tokenizer.batch_decode(val_res['input_ids'])]
  Errors = val_res.query('label!=pred')
  display(Errors)
  '''

  #モデルの保存
  tokenizer.save_pretrained(datadir + 'bert/' + 'BERT_MODEL_' + str(i) + '/')
  model.save_pretrained(datadir + 'bert/' + 'BERT_MODEL_' + str(i) + '/')

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

cuda:0
*** CV 0 started


  0%|          | 0/10 [00:00<?, ?it/s]

  epoch 0 total loss : 14.265589952468872
  epoch 1 total loss : 11.823225200176239
  epoch 2 total loss : 9.339455634355545
  epoch 3 total loss : 7.3647948652505875
  epoch 4 total loss : 5.803341567516327
  epoch 5 total loss : 4.830168694257736
  epoch 6 total loss : 4.323628805577755
  epoch 7 total loss : 3.8065792098641396
  epoch 8 total loss : 3.658075336366892
  epoch 9 total loss : 3.381508521735668


...

  epoch 0 total loss : 14.626262426376343
  epoch 1 total loss : 12.421839892864227
  epoch 2 total loss : 10.20577558875084
  epoch 3 total loss : 8.24857759475708
  epoch 4 total loss : 6.701031759381294
  epoch 5 total loss : 5.545079797506332
  epoch 6 total loss : 4.521889992058277
  epoch 7 total loss : 3.9934447556734085
  epoch 8 total loss : 3.6578828170895576
  epoch 9 total loss : 3.431804023683071


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

cuda:0
*** CV 4 started


  0%|          | 0/10 [00:00<?, ?it/s]

  epoch 0 total loss : 14.229689240455627
  epoch 1 total loss : 12.260186672210693
  epoch 2 total loss : 9.902302086353302
  epoch 3 total loss : 8.049137324094772
  epoch 4 total loss : 6.692678242921829
  epoch 5 total loss : 5.850645646452904
  epoch 6 total loss : 5.233824484050274
  epoch 7 total loss : 4.952340193092823
  epoch 8 total loss : 4.6574667282402515
  epoch 9 total loss : 4.356000768020749
